In [49]:
import pandas as pd
from Bio import SeqIO

In [ ]:
pacbio = pd.read_csv('../../../data/pacbio_protein_clusters.tsv', sep = '\t')
pacbio['accessions'] = pacbio['pb_accs'].str.split('|')
pacbio['base_acc'] = pacbio['accessions'].apply(lambda x : x[0])
orf = pd.read_csv('../../../data/orf-testset-fraction16.csv')

In [40]:
def get_total(accessions, orf_dict):
    total = 0
    for acc in accessions:
        total += orf_dict[acc]
    return total


In [41]:
fl_dict = pd.Series(orf.FL.values,index=orf.pb_acc).to_dict()
cpm_dict = pd.Series(orf.CPM.values,index=orf.pb_acc).to_dict()

In [42]:
pacbio['FL'] = pacbio['accessions'].apply(lambda accs : get_total(accs, fl_dict))
pacbio['CPM'] = pacbio['accessions'].apply(lambda accs : get_total(accs, cpm_dict))

In [72]:
seqs = SeqIO.parse(open('../../../data/pacbio_protein_clusters.fasta'), 'fasta')

In [73]:
with open("../../../data/pacbio_clusters.fasta", "w") as ofile:
    for entry in seqs:
        seq = str(entry.seq)
        pb_acc = entry.id
        pb_row = pacbio[pacbio['pb_accs'] == pb_acc].iloc[0]
        base_acc = pb_row['base_acc']
        gene = orf[orf['pb_acc'] == base_acc].iloc[0]['gene']
        ofile.write(f">pb|{base_acc}|fullname GN={gene}\n{seq}\n")
        

In [76]:
pacbio = pacbio[['pb_accs', 'base_acc', 'FL', 'CPM']]
pacbio.to_csv("../../../data/pacbio_clusters.tsv", sep = '\t', index = False)

,protein_sequence,pb_accs,base_acc,FL,CPM
0,MKELGIWEPLAVKLQTYKTAVETAVLLLRIDDIVSGHKKKGDDQSR...,PB.1091.1,PB.1091.1,2,0.669420
1,MMLNIINSSITTKAISRWSSLACNIALDAVKMVQFEENGRKEIDIK...,PB.1091.10|PB.1091.19,PB.1091.10,34,11.380147
2,MMKMLLDPMGGIVMTNDGNAILREIQVQHPAAKSMIEISRTQDEEV...,PB.1091.2|PB.1091.9|PB.1091.11|PB.1091.12|PB.1...,PB.1091.2,51,17.070221
3,MMGHRPVLVLSQNTKRESGRKVQSGNINAAKTIADIIRTCLGPKSM...,PB.1091.13,PB.1091.13,24,8.033045
4,MMGHRPVLVLSQNTKRESGRKVQSGNINAAKTIADIIRTCLGPKSM...,PB.1091.14,PB.1091.14,4,1.338841
...,...,...,...,...,...
349,MVYMFQYDSTHGKFHGTVKAENGKLVINGNPITIFQERDPSKIKWG...,PB.9726.27|PB.9726.28,PB.9726.27,117,39.161096
350,MEKAGAHLQGGAKRVIISAPSADAPMFVMGVNHEKYDNSLKIISNA...,PB.9726.29|PB.9726.31,PB.9726.29,81,27.111528
351,MEEMCFGEATRMVWLPWVYGNLVSLNMVLSPSPPHPHRRDPSKIKW...,PB.9726.30,PB.9726.30,2,0.669420
352,MFVMGVNHEKYDNSLKIISNASCTTNCLAPLAKVIHDNFGIVEGLM...,PB.9726.32,PB.9726.32,35,11.714858
